In [4]:
import fastf1
from fastf1 import plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.ticker import FuncFormatter, MultipleLocator
import json

# --- FastF1 Cache Configuration ---
# Creates a cache directory for FastF1 to store downloaded data, speeding up future loads.
CACHE_DIR = './fastf1_cache'
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)
fastf1.Cache.enable_cache(CACHE_DIR)
print(f"FastF1 cache enabled at: {os.path.abspath(CACHE_DIR)}")

# --- Base Directory for Processed Data (for saving plots if needed) ---
# This is where generated plots might be stored if plt.savefig() is used.
# The data itself will be loaded directly from FastF1 sessions.
BASE_PLOT_DIR = './f1_plots'
if not os.path.exists(BASE_PLOT_DIR):
    os.makedirs(BASE_PLOT_DIR)
print(f"Base directory for saving plots: {os.path.abspath(BASE_PLOT_DIR)}")


plt.style.use('dark_background') 
COMPOUND_COLORS = fastf1.plotting.COMPOUND_COLORS

FastF1 cache enabled at: c:\Users\mpoli\Desktop\UFOP\8 PERIODO\PAE\Trabalho\fastf1_cache
Base directory for saving plots: c:\Users\mpoli\Desktop\UFOP\8 PERIODO\PAE\Trabalho\f1_plots


C:\Users\mpoli\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fastf1\plotting\__init__.py:84: FutureWarning: INCOMPATIBLE with 2025 season! COMPOUND_COLORS is deprecated and will be removed in a future version.
  warnings.warn(f"INCOMPATIBLE with 2025 season! {name} is deprecated "


In [5]:
def format_laptime_mmssms(seconds: float) -> str:
    if pd.isna(seconds):
        return ""
    minutes = int(seconds // 60)
    remaining_seconds = seconds % 60
    return f"{minutes:02d}:{remaining_seconds:06.3f}"

def get_clean_laps(session_laps: pd.DataFrame, TARGET_YEAR: int, race_name: str) -> pd.DataFrame:
    # Return where lap[‘IsAccurate’] is True
    # Fastf1 has a column 'IsAccurate' that indicates if the lap is accurate and seems to work well
    # First laps are often not accurate, so we filter them out, and also exclude laps with no time and pit in/out laps
    clean_laps = session_laps[session_laps['IsAccurate'] == True]
    #checking if clean_laps has 'pirelliCompound' row
    if 'pirelliCompound' is clean_laps.columns:
        return clean_laps.reset_index(drop=True)
    else:
        # Acessing the json to correlate soft, medium and hard with the compounds
        json_path = 'compounds.json'
        with open(json_path, 'r') as file:
            compounds_data = json.load(file)
    
        year_str = str(TARGET_YEAR)
    
        if year_str in compounds_data['data']:
            compounds_for_year = compounds_data['data'][year_str]
            gp_compound_mapping = compounds_for_year.get(race_name)

            if gp_compound_mapping:
                clean_laps['pirelliCompound'] = clean_laps['Compound'].map(gp_compound_mapping)
            
                clean_laps['pirelliCompound'] = clean_laps['pirelliCompound'].fillna(clean_laps['Compound'])
                print(f" 'pirelliCompound' column created for {race_name} {TARGET_YEAR}.")

            else:
                print(f" Warning: No compound mapping found in JSON for '{race_name}' in year {TARGET_YEAR}. 'pirelliCompound' not created.")
                clean_laps['pirelliCompound'] = clean_laps['Compound']
        else:
            print(f" Warning: Year {TARGET_YEAR} not found in 'compounds.json'. 'pirelliCompound' not created.")
            clean_laps['pirelliCompound'] = clean_laps['Compound']
    return clean_laps.reset_index(drop=True)

    


<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\mpoli\AppData\Local\Temp\ipykernel_21056\2982951129.py:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if 'pirelliCompound' is clean_laps.columns:


In [6]:
target_gp_name = 'Hungarian Grand Prix' 
start_year_analysis = 2022
end_year_analysis = 2025

all_results_data_by_year = {}
all_laps_data_by_year = {}
all_stints_data_by_year = {}
all_weather_data_by_year = {}
date = {}


print(f"\n--- Starting comprehensive data collection for {target_gp_name} from {start_year_analysis} to {end_year_analysis-1} ---")

for year in range(start_year_analysis, end_year_analysis):
    print(f"\n--- Collecting data for {target_gp_name} in Season: {year} ---")
    schedule = fastf1.get_event_schedule(year)
    target_event = schedule[schedule['EventName'].str.contains(target_gp_name, case=False, na=False)]

    if target_event.empty:
        print(f"  {target_gp_name} not found in {year} schedule, skipping.")
        continue
    target_race_round_row = target_event[target_event['EventFormat'].isin(['conventional', 'sprint'])]
        
    if target_race_round_row.empty:
        print(f"  No 'conventional' or 'sprint' race event found for {target_gp_name} in {year}, skipping.")
        continue

    round_num = target_race_round_row['RoundNumber'].iloc[0]
    actual_event_name = target_race_round_row['EventName'].iloc[0] 

    print(f"  Found {actual_event_name} as Round {round_num} in {year}. Loading session...")

    session = fastf1.get_session(year, round_num, 'R') 
    session.load(laps=True, telemetry=True, weather=True, messages=True)

    laps_df = session.laps.copy()
    results_df = session.results.copy()
    
    if 'LapTime_seconds' not in laps_df.columns:
        laps_df['LapTime_seconds'] = laps_df['LapTime'].dt.total_seconds()

    race_name = session.event['EventName']
    json_path = 'compounds.json'
    with open(json_path, 'r') as file:
        compounds_data = json.load(file)
    
    year_str = str(year)
    
    if year_str in compounds_data['data']:
        compounds_for_year = compounds_data['data'][year_str]
        gp_compound_mapping = compounds_for_year.get(target_gp_name)

        if gp_compound_mapping:
            laps_df['pirelliCompound'] = laps_df['Compound'].map(gp_compound_mapping)
            
            laps_df['pirelliCompound'] = laps_df['pirelliCompound'].fillna(laps_df['Compound'])
            
            print(f" 'pirelliCompound' column created for {target_gp_name} {year}.")

        else:
            print(f" Warning: No compound mapping found in JSON for '{target_gp_name}' in year {year}. 'pirelliCompound' not created.")
    else:
        print(f" Warning: Year {year} not found in 'compounds.json'. 'pirelliCompound' not created.")
        laps_df['pirelliCompound'] = laps_df['Compound']

    all_results_data_by_year[year] = results_df
    all_laps_data_by_year[year] = laps_df
    all_stints_data_by_year[year] = laps_df[["Driver", "Stint", "Compound", "LapNumber","pirelliCompound"]]
    all_stints_data_by_year[year] = all_stints_data_by_year[year].groupby(["Driver", "Stint", "Compound", "pirelliCompound"])
    all_stints_data_by_year[year] = all_stints_data_by_year[year].count().reset_index() 
    all_stints_data_by_year[year] = all_stints_data_by_year[year].rename(columns={"LapNumber": "StintLength"})
    all_weather_data_by_year[year] = session.weather_data
    date[year]= session.date
    
    print(f"Session data loaded for: {race_name} {year} Round {round_num}")
    print(f"Total laps: {len(laps_df)}, Total results: {len(results_df)}")

core           INFO 	Loading data for Hungarian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...



--- Starting comprehensive data collection for Hungarian Grand Prix from 2022 to 2024 ---

--- Collecting data for Hungarian Grand Prix in Season: 2022 ---
  Found Hungarian Grand Prix as Round 13 in 2022. Loading session...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '63', '55', '11', '16', '4', '14', '31', '5', '18', '10', '24', '47', '3', '20', '23', '6', '22', '77']
core           INFO 	Loading data for Hungarian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


 'pirelliCompound' column created for Hungarian Grand Prix 2022.
Session data loaded for: Hungarian Grand Prix 2022 Round 13
Total laps: 1383, Total results: 20

--- Collecting data for Hungarian Grand Prix in Season: 2023 ---
  Found Hungarian Grand Prix as Round 11 in 2023. Loading session...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '11', '44', '81', '63', '16', '55', '14', '18', '23', '77', '3', '27', '22', '24', '20', '2', '31', '10']
core           INFO 	Loading data for Hungarian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


 'pirelliCompound' column created for Hungarian Grand Prix 2023.
Session data loaded for: Hungarian Grand Prix 2023 Round 11
Total laps: 1252, Total results: 20

--- Collecting data for Hungarian Grand Prix in Season: 2024 ---
  Found Hungarian Grand Prix as Round 13 in 2024. Loading session...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '4', '44', '16', '1', '55', '11', '63', '22', '18', '14', '3', '27', '23', '20', '77', '2', '31', '24', '10']


 'pirelliCompound' column created for Hungarian Grand Prix 2024.
Session data loaded for: Hungarian Grand Prix 2024 Round 13
Total laps: 1355, Total results: 20


In [7]:
top_n_strategies = 5

all_strategies = []

print(f"\n--- Analyzing Most Used Strategies for {target_gp_name} across years ---")

for year, data_for_year in all_stints_data_by_year.items():
    if not data_for_year.empty: 
        stints_df = data_for_year.copy()
        
        driver_strategies = stints_df.groupby('Driver').agg(
            compound_sequence=('pirelliCompound', lambda x: tuple(x.tolist())) # Get ordered tuple of compounds
        ).reset_index()

        driver_strategies['strategy_string'] = driver_strategies['compound_sequence'].apply(lambda x: "-".join(x))
        
        # Add year to the strategy for context (useful for df_all_strategies)
        driver_strategies['year'] = year
        
        all_strategies.append(driver_strategies)
    else:
        print(f"  No stints data found for {year} {target_gp_name}, skipping.")

df_all_strategies = pd.concat(all_strategies, ignore_index=True)


# Count the frequency of each unique strategy string across all years
strategy_counts = df_all_strategies['strategy_string'].value_counts().reset_index()
strategy_counts.columns = ['Strategy', 'Count']

# Get the top N most frequent strategies
top_strategies = strategy_counts.head(top_n_strategies)
print(f"\nTop {top_n_strategies} Most Used Strategies for {target_gp_name}:")
display(top_strategies)


--- Analyzing Most Used Strategies for Hungarian Grand Prix across years ---

Top 5 Most Used Strategies for Hungarian Grand Prix:


,Strategy,Count
0,C4-C3-C3,17
1,C4-C3-C4,12
2,C5-C3-C3,4
3,C3-C4-C4,3
4,C3-C2,3


In [8]:
all_strategy_score = {}
merged_data = {}

C = 0
for year in all_results_data_by_year.keys():
  data_for_year = all_results_data_by_year[year]
  finished_drivers_results = data_for_year[
     ~data_for_year['Status'].str.contains(
        'Retired', 
        case=False, na=False, regex=True
    )
  ].copy()
  finished_drivers_results['Positions_Gained_Lost'] = finished_drivers_results['GridPosition'] - finished_drivers_results['Position']
  finished_drivers_results['Strategy_Score'] = finished_drivers_results['Points'] + C * finished_drivers_results['Positions_Gained_Lost']
  all_strategy_score[year] = finished_drivers_results

  stints_df = all_stints_data_by_year[year]
  driver_strategies = stints_df.groupby('Driver').agg(
            compound_sequence=('pirelliCompound', lambda x: tuple(x.tolist()))
        ).reset_index()
  driver_strategies['strategy_string'] = driver_strategies['compound_sequence'].apply(lambda x: "-".join(x))
  merged_data[year] = pd.merge(driver_strategies, all_strategy_score[year], left_on='Driver', right_on='Abbreviation')
  

In [9]:
data_filtered_by_year = {}
median_weather_by_year = {}
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge 
from sklearn.metrics import r2_score, mean_absolute_error


for year, df_data in merged_data.items():
    print(f"\n--- Processando dados para o ano {year} ---")

    if df_data.empty:
        print(f"   Dados de merged_data para o ano {year} estão vazios, pulando.")
        continue

    df_year_copy = df_data.copy()
    df_year_copy['year'] = year

    cleaned_df = df_year_copy.dropna(subset=[
        'strategy_string', 'Driver', 'GridPosition', 'Position',
        'Strategy_Score', 'Positions_Gained_Lost'
    ])
    
    data_filtered_by_year[year] = cleaned_df
    
    if year in all_weather_data_by_year and not all_weather_data_by_year[year].empty:
        median_weather_by_year[year] = all_weather_data_by_year[year].select_dtypes(include=np.number).median()
    else:
        print(f"   Dados de tempo para o ano {year} não encontrados ou vazios.")

final_races_df = pd.concat(data_filtered_by_year.values(), ignore_index=True)

median_weather_df = pd.DataFrame.from_dict(median_weather_by_year, orient='index')
median_weather_df.index.name = 'year'
median_weather_df.reset_index(inplace=True)

final_df = pd.merge(final_races_df, median_weather_df, on='year', how='left', suffixes=('', '_weather'))

print("--- Filtrando pelas Top Estratégias ---")
top_strategies_list = top_strategies['Strategy'].tolist()
print(f"Estratégias a serem utilizadas: {top_strategies_list}")
df_filtered = final_df[final_df['strategy_string'].isin(top_strategies_list)].copy()
print(f"O número de amostras foi reduzido de {len(final_df)} para {len(df_filtered)} após a filtragem.")


print("\n--- Iniciando Engenharia de Features (versão estendida para até 5 stints) ---")

df_engineered = df_filtered.copy()
compound_list = df_engineered['strategy_string'].str.split('-')

df_engineered['num_stints'] = compound_list.str.len()

df_engineered['start_compound'] = compound_list.str.get(0)

df_engineered['second_stint_compound'] = compound_list.str.get(1)

df_engineered['third_stint_compound'] = compound_list.str.get(2)

df_engineered['fourth_stint_compound'] = compound_list.str.get(3)

df_engineered['fifth_stint_compound'] = compound_list.str.get(4)


df_engineered['second_stint_compound'].fillna('None', inplace=True)
df_engineered['third_stint_compound'].fillna('None', inplace=True)
df_engineered['fourth_stint_compound'].fillna('None', inplace=True)
df_engineered['fifth_stint_compound'].fillna('None', inplace=True)
print("Novas features criadas para até 5 stints.")

target = 'Strategy_Score'

numerical_features = ['GridPosition', 'TrackTemp', 'num_stints'] 

categorical_features = ['start_compound', 
                        'second_stint_compound', 
                        'third_stint_compound', 
                        'fourth_stint_compound', 
                        'fifth_stint_compound']

model_df = df_engineered[numerical_features + categorical_features + [target]].dropna().copy()
display(model_df)
X = model_df.drop(columns=target)
y = model_df[target]
print("\n--- Preparando o Pipeline de Modelagem ---")

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1.0, random_state=42))
])
print("Pipeline construído com sucesso.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(f"\nDados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")
model_pipeline.fit(X_train, y_train)
print("Modelo treinado com sucesso!")
y_pred = model_pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\n--- Resultados Finais da Avaliação (Stints Estendidos) ---")
print(f"R² : {r2:.3f}")
print(f"MAE : {mae:.3f}")

try:
    encoded_cat_features = model_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
    all_feature_names = numerical_features + list(encoded_cat_features)
    coefficients = model_pipeline.named_steps['regressor'].coef_
    coef_df = pd.DataFrame({'Feature': all_feature_names, 'Coefficient': coefficients})
    coef_df['Abs_Coefficient'] = abs(coef_df['Coefficient'])
    coef_df_sorted = coef_df.sort_values(by='Abs_Coefficient', ascending=False)
    print("\n--- Features Mais Influentes (de acordo com o modelo) ---")
    display(coef_df_sorted.head(10).drop(columns='Abs_Coefficient'))
except Exception as e:
    print(f"\nNão foi possível gerar a análise de coeficientes: {e}")


--- Processando dados para o ano 2022 ---

--- Processando dados para o ano 2023 ---

--- Processando dados para o ano 2024 ---
--- Filtrando pelas Top Estratégias ---
Estratégias a serem utilizadas: ['C4-C3-C3', 'C4-C3-C4', 'C5-C3-C3', 'C3-C4-C4', 'C3-C2']
O número de amostras foi reduzido de 57 para 39 após a filtragem.

--- Iniciando Engenharia de Features (versão estendida para até 5 stints) ---
Novas features criadas para até 5 stints.


C:\Users\mpoli\AppData\Local\Temp\ipykernel_21056\407632779.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_engineered['second_stint_compound'].fillna('None', inplace=True)
C:\Users\mpoli\AppData\Local\Temp\ipykernel_21056\407632779.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

,GridPosition,TrackTemp,num_stints,start_compound,second_stint_compound,third_stint_compound,fourth_stint_compound,fifth_stint_compound,Strategy_Score
1,6.0,26.70,2,C3,C2,None,None,None,4.0
2,8.0,26.70,2,C3,C2,None,None,None,0.0
3,0.0,26.70,3,C4,C3,C4,None,None,0.0
10,5.0,26.70,2,C3,C2,None,None,None,2.0
11,11.0,26.70,3,C4,C3,C3,None,None,10.0
13,1.0,26.70,3,C4,C3,C3,None,None,15.0
15,14.0,26.70,3,C4,C3,C4,None,None,0.0
17,10.0,26.70,3,C4,C3,C3,None,None,25.0
18,18.0,26.70,3,C4,C3,C3,None,None,1.0
20,16.0,48.65,3,C4,C3,C3,None,None,0.0



--- Preparando o Pipeline de Modelagem ---
Pipeline construído com sucesso.

Dados divididos: 33 para treino, 6 para teste.
Modelo treinado com sucesso!

--- Resultados Finais da Avaliação (Stints Estendidos) ---
R² : 0.412
MAE : 3.325

--- Features Mais Influentes (de acordo com o modelo) ---


,Feature,Coefficient
0,GridPosition,-4.309925
2,num_stints,2.662881
8,second_stint_compound_C4,2.163339
10,third_stint_compound_C4,1.794510
7,second_stint_compound_C3,-1.397814
3,start_compound_C3,1.397814
9,third_stint_compound_C3,-1.028985
5,start_compound_C5,-0.874171
6,second_stint_compound_C2,-0.765524
11,third_stint_compound_None,-0.765524


In [10]:
cenario_temp = model_df['TrackTemp'].mean()

print("--- Cenário de Simulação Definido (Condições Médias) ---")
print(f"Temperatura da Pista Média: {cenario_temp:.2f}°C")

# Lista de estratégias que queremos simular
novas_estrategias = [
    'C3-C1-C1',  # macio-duro-duro
    'C2-C2-C1',  # medio-duro-duro
    'C1-C2-C2',  # duro-medio-medio
    'C2-C1-C2',  # medio-duro-medio
    'C1-C1-C2',  # duro-duro-medio
    'C2-C1-C1', # medio-duro-duro
    'C3-C2-C3-C3', # macio-medio-macio-macio
    'C2-C2-C2-C1',  # medio-medio-medio-duro
]

# Criamos um DataFrame com essas estratégias
df_hipotetico = pd.DataFrame({'strategy_string': novas_estrategias})

# Adicionamos as colunas do nosso cenário fixo
df_hipotetico['GridPosition'] = 10
df_hipotetico['TrackTemp'] = cenario_temp

print("\n--- DataFrame com Estratégias Hipotéticas Criado ---")
display(df_hipotetico)

--- Cenário de Simulação Definido (Condições Médias) ---
Temperatura da Pista Média: 42.40°C

--- DataFrame com Estratégias Hipotéticas Criado ---


,strategy_string,GridPosition,TrackTemp
0,C3-C1-C1,10,42.401282
1,C2-C2-C1,10,42.401282
2,C1-C2-C2,10,42.401282
3,C2-C1-C2,10,42.401282
4,C1-C1-C2,10,42.401282
5,C2-C1-C1,10,42.401282
6,C3-C2-C3-C3,10,42.401282
7,C2-C2-C2-C1,10,42.401282


In [11]:
# Esta é a mesma lógica de engenharia de features que você já criou

print("--- Aplicando Engenharia de Features nos Dados Hipotéticos ---")
compound_list = df_hipotetico['strategy_string'].str.split('-')

df_hipotetico['num_stints'] = compound_list.str.len()
df_hipotetico['start_compound'] = compound_list.str.get(0)
df_hipotetico['second_stint_compound'] = compound_list.str.get(1).fillna('None')
df_hipotetico['third_stint_compound'] = compound_list.str.get(2).fillna('None')
df_hipotetico['fourth_stint_compound'] = compound_list.str.get(3).fillna('None')
df_hipotetico['fifth_stint_compound'] = compound_list.str.get(4).fillna('None')

print("Engenharia de features aplicada.")

# O DataFrame hipotético agora tem todas as colunas que o pipeline espera.
# Note que a ordem das colunas não importa, o ColumnTransformer cuida disso.
X_hipotetico = df_hipotetico.drop(columns=['strategy_string']) # Removemos a string original

# Usamos o pipeline treinado para prever os scores
pontuacoes_previstas = model_pipeline.predict(X_hipotetico)

# Adicionamos as previsões ao nosso DataFrame para uma análise fácil
df_hipotetico['Score_Previsto'] = pontuacoes_previstas

# Ordenamos para ver qual estratégia o modelo considera a melhor
df_simulacao_final = df_hipotetico.sort_values(by='Score_Previsto', ascending=False)

print("\n\n--- RESULTADO DA SIMULAÇÃO ---")
print("Estratégias ranqueadas pela pontuação prevista pelo modelo:")
display(df_simulacao_final[['strategy_string', 'Score_Previsto']])

--- Aplicando Engenharia de Features nos Dados Hipotéticos ---
Engenharia de features aplicada.


--- RESULTADO DA SIMULAÇÃO ---
Estratégias ranqueadas pela pontuação prevista pelo modelo:


,strategy_string,Score_Previsto
6,C3-C2-C3-C3,16.859863
7,C2-C2-C2-C1,16.491034
0,C3-C1-C1,9.391527
3,C2-C1-C2,7.993712
5,C2-C1-C1,7.993712
4,C1-C1-C2,7.993712
2,C1-C2-C2,7.228188
1,C2-C2-C1,7.228188
